<a href="https://colab.research.google.com/github/bokuwakira1411/Data-mining---ex2-3/blob/main/exercise_2%2B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Xử lý dữ liệu training set

In [55]:

df_train = pd.read_csv("/content/drive/My Drive/Dataset/labeled_train.csv")
df_train.head()

,Date,Time,Open,High,Low,Close,Volume,Label
0,2011-01-03,01:00,1415.71,1416.54,1415.10,1415.63,1035,SELL
1,2011-01-03,02:00,1415.64,1416.43,1414.90,1415.98,736,SELL
2,2011-01-03,03:00,1415.95,1417.89,1414.59,1416.54,922,SELL
3,2011-01-03,04:00,1416.51,1418.64,1415.25,1416.75,1060,SELL
4,2011-01-03,05:00,1416.76,1420.00,1416.76,1419.48,838,SELL


In [56]:
df_train.isnull().sum()

,0
Date,0
Time,0
Open,0
High,0
Low,0
Close,0
Volume,0
Label,0


In [57]:
df_train['Date Time'] = pd.to_datetime(df_train['Date'] + ' ' + df_train['Time'])
df_train = df_train.set_index('Date Time').drop(columns=['Date', 'Time'])
df_train.head()

,Open,High,Low,Close,Volume,Label
Date Time,,,,,,
2011-01-03 01:00:00,1415.71,1416.54,1415.10,1415.63,1035,SELL
2011-01-03 02:00:00,1415.64,1416.43,1414.90,1415.98,736,SELL
2011-01-03 03:00:00,1415.95,1417.89,1414.59,1416.54,922,SELL
2011-01-03 04:00:00,1416.51,1418.64,1415.25,1416.75,1060,SELL
2011-01-03 05:00:00,1416.76,1420.00,1416.76,1419.48,838,SELL


### Sample dữ liệu bị thiếu

In [58]:
# Xử lý thiếu dữ liệu bằng phương pháp forward fill cho 10 phút ngay sau, nếu vẫn cần sample tiếp thì sử dụng interpolate time.

df_train['source'] = 'original'     # Thêm cột đánh dấu là dữ liệu sample hay original
df_train_resampled = df_train.resample('1T').ffill(limit=10)
df_train_resampled = df_train_resampled.interpolate(method='time')

# # Đánh dấu các dòng nào là original (có trong index gốc)
df_train_resampled['source'] = df_train_resampled.index.isin(df_train.index)
df_train_resampled['source'] = df_train_resampled['source'].map({True: 'original', False: 'sampled'})

df_train_resampled.iloc[55:65]

<ipython-input-58-a662acf6b23f>:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_train_resampled = df_train.resample('1T').ffill(limit=10)
<ipython-input-58-a662acf6b23f>:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_train_resampled = df_train_resampled.interpolate(method='time')


,Open,High,Low,Close,Volume,Label,source
Date Time,,,,,,,
2011-01-03 01:55:00,1415.6470,1416.4410,1414.920,1415.945,765.90,NaN,sampled
2011-01-03 01:56:00,1415.6456,1416.4388,1414.916,1415.952,759.92,NaN,sampled
2011-01-03 01:57:00,1415.6442,1416.4366,1414.912,1415.959,753.94,NaN,sampled
2011-01-03 01:58:00,1415.6428,1416.4344,1414.908,1415.966,747.96,NaN,sampled
2011-01-03 01:59:00,1415.6414,1416.4322,1414.904,1415.973,741.98,NaN,sampled
2011-01-03 02:00:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,original
2011-01-03 02:01:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled
2011-01-03 02:02:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled
2011-01-03 02:03:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled


In [59]:
print(df_train.shape)
print(df_train_resampled.shape)

(3446549, 7)
(5257098, 7)


### Tính toán để gán Label cho dữ liệu vừa sample được

In [60]:
# Gán label cho dữ liệu đã sample (chỉ dữ liệu sử dụng interpolate time,
# vì dữ liệu sample 10 phút ban đầu bằng bfill chỉ điền giống dữ liệu gần nhất nên không cần gán lại Label)

delta_minutes = 30
threshold = 0.000505
df_train_resampled['price_future'] = df_train_resampled['Close'].shift(-delta_minutes)

mask_no_label = df_train_resampled['Label'].isna()

df_train_resampled.loc[mask_no_label, 'Label'] = np.select(
    [
        df_train_resampled.loc[mask_no_label, 'price_future'] > df_train_resampled.loc[mask_no_label, 'Close'] * (1 + threshold),
        df_train_resampled.loc[mask_no_label, 'price_future'] < df_train_resampled.loc[mask_no_label, 'Close'] * (1 - threshold)
    ],
    ['BUY', 'SELL'],
    default='HOLD'
)
df_train_resampled.drop(columns='price_future', inplace= True)
df_train_resampled.iloc[55:65]

,Open,High,Low,Close,Volume,Label,source
Date Time,,,,,,,
2011-01-03 01:55:00,1415.6470,1416.4410,1414.920,1415.945,765.90,HOLD,sampled
2011-01-03 01:56:00,1415.6456,1416.4388,1414.916,1415.952,759.92,HOLD,sampled
2011-01-03 01:57:00,1415.6442,1416.4366,1414.912,1415.959,753.94,HOLD,sampled
2011-01-03 01:58:00,1415.6428,1416.4344,1414.908,1415.966,747.96,HOLD,sampled
2011-01-03 01:59:00,1415.6414,1416.4322,1414.904,1415.973,741.98,HOLD,sampled
2011-01-03 02:00:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,original
2011-01-03 02:01:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled
2011-01-03 02:02:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled
2011-01-03 02:03:00,1415.6400,1416.4300,1414.900,1415.980,736.00,SELL,sampled


### Tính toán các cột dữ liệu thành % thay đổi so với thời điểm ngay trước

In [61]:
change_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

# Tính phần trăm thay đổi (returns) cho từng cột
for col in change_cols:
    df_train_resampled[f'{col}_return'] = df_train_resampled[col].pct_change()

# Loại bỏ dòng đầu tiên (vì không có dữ liệu để tính % thay đổi)
df_train_resampled = df_train_resampled.dropna(subset=[f'{col}_return' for col in change_cols])

df_train_resampled.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

df_train_resampled.tail()

,Label,source,Open_return,High_return,Low_return,Close_return,Volume_return
Date Time,,,,,,,
2020-12-31 19:13:00,BUY,original,-0.000169,0.000360,-0.000111,0.000529,0.417722
2020-12-31 19:14:00,BUY,original,0.000529,0.000333,0.000672,0.000143,-0.312500
2020-12-31 19:15:00,BUY,original,0.000095,-0.000227,-0.000058,-0.000137,-0.116883
2020-12-31 19:16:00,BUY,original,-0.000090,-0.000079,-0.000248,-0.000312,0.102941
2020-12-31 19:17:00,BUY,original,-0.000312,-0.000270,-0.000143,0.000005,-0.040000


### Chuẩn hoá dữ liệu Z-score, đưa dữ liệu về phân phối có trung bình = 0 và độ lệch chuẩn = 1

In [62]:
from sklearn.preprocessing import StandardScaler

feature_cols = ['Open_return', 'High_return', 'Low_return', 'Close_return',	'Volume_return']
scaler = StandardScaler()

# Fit và transform
df_train_resampled[feature_cols] = scaler.fit_transform(df_train_resampled[feature_cols])
df_train_resampled.tail()

,Label,source,Open_return,High_return,Low_return,Close_return,Volume_return
Date Time,,,,,,,
2020-12-31 19:13:00,BUY,original,-0.774661,1.822141,-0.529054,2.422634,0.181306
2020-12-31 19:14:00,BUY,original,2.419735,1.687525,3.197280,0.653501,-0.323131
2020-12-31 19:15:00,BUY,original,0.435026,-1.152055,-0.277143,-0.629918,-0.187999
2020-12-31 19:16:00,BUY,original,-0.411521,-0.402217,-1.183024,-1.429166,-0.036144
2020-12-31 19:17:00,BUY,original,-1.427456,-1.366739,-0.679935,0.023863,-0.134888


## Xử lý dữ liệu development set

In [63]:
df_dev = pd.read_csv("/content/drive/My Drive/Dataset/labeled_dev.csv")
df_dev.head()

,Date,Time,Open,High,Low,Close,Volume,Label
0,2021-01-04,00:05,1909.07,1909.33,1907.95,1908.40,236,BUY
1,2021-01-04,00:06,1908.36,1909.18,1907.82,1908.68,193,BUY
2,2021-01-04,00:07,1908.67,1908.80,1908.18,1908.27,113,BUY
3,2021-01-04,00:08,1908.27,1908.51,1907.84,1908.15,103,BUY
4,2021-01-04,00:09,1908.19,1908.88,1907.73,1908.77,165,BUY


In [64]:
df_dev.isnull().sum()

,0
Date,0
Time,0
Open,0
High,0
Low,0
Close,0
Volume,0
Label,0


In [65]:
df_dev['Date Time'] = pd.to_datetime(df_dev['Date'] + ' ' + df_dev['Time'])
df_dev = df_dev.set_index('Date Time').drop(columns=['Date', 'Time'])
df_dev.head()

,Open,High,Low,Close,Volume,Label
Date Time,,,,,,
2021-01-04 00:05:00,1909.07,1909.33,1907.95,1908.40,236,BUY
2021-01-04 00:06:00,1908.36,1909.18,1907.82,1908.68,193,BUY
2021-01-04 00:07:00,1908.67,1908.80,1908.18,1908.27,113,BUY
2021-01-04 00:08:00,1908.27,1908.51,1907.84,1908.15,103,BUY
2021-01-04 00:09:00,1908.19,1908.88,1907.73,1908.77,165,BUY


In [66]:
df_dev.shape

(350984, 6)

### Sample dữ liệu

In [67]:
# Xử lý thiếu dữ liệu bằng phương pháp forward fill cho 10 phút ngay sau, nếu vẫn cần sample tiếp thì sử dụng interpolate time.

df_dev['source'] = 'original'     # Thêm cột đánh dấu là dữ liệu sample hay original
df_dev_resampled = df_dev.resample('1T').ffill(limit=10)
df_dev_resampled = df_dev_resampled.interpolate(method='time')

# # Đánh dấu các dòng nào là original (có trong index gốc)
df_dev_resampled['source'] = df_dev_resampled.index.isin(df_dev.index)
df_dev_resampled['source'] = df_dev_resampled['source'].map({True: 'original', False: 'sampled'})

df_dev_resampled.head()

<ipython-input-67-afddb2451a97>:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_dev_resampled = df_dev.resample('1T').ffill(limit=10)
<ipython-input-67-afddb2451a97>:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_dev_resampled = df_dev_resampled.interpolate(method='time')


,Open,High,Low,Close,Volume,Label,source
Date Time,,,,,,,
2021-01-04 00:05:00,1909.07,1909.33,1907.95,1908.40,236.0,BUY,original
2021-01-04 00:06:00,1908.36,1909.18,1907.82,1908.68,193.0,BUY,original
2021-01-04 00:07:00,1908.67,1908.80,1908.18,1908.27,113.0,BUY,original
2021-01-04 00:08:00,1908.27,1908.51,1907.84,1908.15,103.0,BUY,original
2021-01-04 00:09:00,1908.19,1908.88,1907.73,1908.77,165.0,BUY,original


In [68]:
df_dev_resampled.shape

(520993, 7)

### Gán nhãn dữ liệu được sample

In [69]:
# Gán label cho dữ liệu đã sample (chỉ dữ liệu sử dụng interpolate time,
# vì dữ liệu sample 10 phút ban đầu bằng bfill chỉ điền giống dữ liệu gần nhất nên không cần gán lại Label)

delta_minutes = 30
threshold = 0.000505
df_dev_resampled['price_future'] = df_dev_resampled['Close'].shift(-delta_minutes)

mask_no_label = df_dev_resampled['Label'].isna()

df_dev_resampled.loc[mask_no_label, 'Label'] = np.select(
    [
        df_dev_resampled.loc[mask_no_label, 'price_future'] > df_dev_resampled.loc[mask_no_label, 'Close'] * (1 + threshold),
        df_dev_resampled.loc[mask_no_label, 'price_future'] < df_dev_resampled.loc[mask_no_label, 'Close'] * (1 - threshold)
    ],
    ['BUY', 'SELL'],
    default='HOLD'
)
df_dev_resampled.drop(columns='price_future', inplace= True)
df_dev_resampled[df_dev_resampled['source']=='sampled'][:5]

,Open,High,Low,Close,Volume,Label,source
Date Time,,,,,,,
2021-01-04 22:56:00,1943.23,1943.23,1943.23,1943.23,1.0,BUY,sampled
2021-01-04 22:57:00,1943.23,1943.23,1943.23,1943.23,1.0,BUY,sampled
2021-01-04 22:58:00,1943.23,1943.23,1943.23,1943.23,1.0,BUY,sampled
2021-01-04 22:59:00,1943.23,1943.23,1943.23,1943.23,1.0,BUY,sampled
2021-01-04 23:00:00,1943.23,1943.23,1943.23,1943.23,1.0,BUY,sampled


### Tính toán % thay đổi

In [70]:
change_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

# Tính phần trăm thay đổi (returns) cho từng cột
for col in change_cols:
    df_dev_resampled[f'{col}_return'] = df_dev_resampled[col].pct_change()

# Loại bỏ dòng đầu tiên (vì không có dữ liệu để tính % thay đổi)
df_dev_resampled = df_dev_resampled.dropna(subset=[f'{col}_return' for col in change_cols])

df_dev_resampled.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

df_dev_resampled.tail()

<ipython-input-70-07a43e6b4964>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev_resampled.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)


,Label,source,Open_return,High_return,Low_return,Close_return,Volume_return
Date Time,,,,,,,
2021-12-31 19:13:00,BUY,original,-0.000033,0.000022,0.000005,0.000088,-0.280000
2021-12-31 19:14:00,BUY,original,0.000082,0.000164,0.000082,0.000153,0.416667
2021-12-31 19:15:00,BUY,original,0.000142,0.000120,0.000164,0.000071,-0.549020
2021-12-31 19:16:00,BUY,original,0.000060,0.000038,0.000022,-0.000038,1.304348
2021-12-31 19:17:00,BUY,original,-0.000022,-0.000126,-0.000142,-0.000099,0.075472


### Chuẩn hoá Z-score

In [71]:
# Dùng scaler đã fit ở train (chỉ transform, không fit lại!)

df_dev_resampled[feature_cols] = scaler.transform(df_dev_resampled[feature_cols])
df_dev_resampled.head()


,Label,source,Open_return,High_return,Low_return,Close_return,Volume_return
Date Time,,,,,,,
2021-01-04 00:06:00,BUY,original,-1.702375,-0.398614,-0.324734,0.671895,-0.233122
2021-01-04 00:07:00,BUY,original,0.743048,-1.009321,0.897937,-0.984593,-0.393598
2021-01-04 00:08:00,BUY,original,-0.959443,-0.770514,-0.848607,-0.288491,-0.168388
2021-01-04 00:09:00,BUY,original,-0.192218,0.982360,-0.274847,1.488402,0.308566
2021-01-04 00:10:00,BUY,original,1.150826,-0.292486,-0.025322,-1.392623,-0.454749
